### SIEM

In [7]:
import requests
import time
import re
from dataclasses import dataclass
from typing import Generator, Optional
from io import IOBase
import os

from dotenv import load_dotenv

load_dotenv()
api_token = os.getenv('API_TOKEN')

In [8]:
def tail_follow(file: IOBase) -> Generator[str, None, None]:
    file.seek(0, 2)
    while True:
        line = file.readline()
        if line:
            yield line
            continue
        time.sleep(1)

In [10]:
def papertrail_tail(token: str) -> Generator[str, None, None]:
    url = "https://papertrailapp.com/api/v1/events/search.json"
    headers = {"X-Papertrail-Token": token}

    tail = False
    min_id = None
    limit = 10000

    while True:
        params = {"limit": limit}
        if tail and min_id:
            params["min_id"] = min_id
        else:
            params["min_time"] = int(time.time()) - 10
        
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        data = r.json()
        tail = data.get("tail", False)
        min_id = data.get("max_id", None)
        yield from (e["message"] for e in data["events"])
        time.sleep(5)

In [11]:
@dataclass
class PacketInfo:
    """ Information about a network packet"""
    timestamp: str
    protocol: str
    src: str
    dst: str
    data: str
    

In [12]:
# TODO Syslog format parsing?

def parse_tcpdump(line: str) -> Optional[PacketInfo]:
    pattern = re.compile(r'(\d+:\d+:\d+\.\d+) (\w+) ([\d\.]+|\w+) > ([\d\.]+|\w+): (.+)$')
    match = pattern.match(line)
    if match:
        return PacketInfo(*list(match.groups()))
    return None

In [ ]:
for line in papertrail_tail(api_token):
    pkt = parse_tcpdump(line)
    if not pkt:
        print("Packet format error")
    else:
        print(f"Packet from {pkt.src} to {pkt.dst}: {pkt.data}")

In [ ]:
# Port scan detection
from datetime import datetime, timedelta

packet_counts = {}

time_window = timedelta(seconds=10)
packet_threshold = 100

start_time = None

for line in papertrail_tail(api_token):
    pkt = parse_tcpdump(line)
    if not pkt:
        print("Packet format error")
    else:
        print(f"Packet from {pkt.src} to {pkt.dst}: {pkt.data}")

        pkt_time = datetime.strptime(pkt.timestamp, "%H:%M:%S.%f")

        if start_time is None or pkt_time - start_time > time_window:
            packet_counts.clear()
            start_time = pkt_time

        packet_counts[pkt.dst] += 1

        
        if packet_counts[pkt.dst] > packet_threshold:
            print(f"Port scan attack detected from: {pkt.src}")

In [ ]:
# Keep track of SYN packets sent to each destination
syn_counts = {}

# Define the time window and packet count threshold
time_window = timedelta(seconds=10)
packet_threshold = 1000

# Keep track of the start time of the current time window
start_time = None

for line in papertrail_tail(api_token):
    pkt = parse_tcpdump(line)
    if not pkt:
        print("Packet format error")
    else:
        print(f"Packet from {pkt.src} to {pkt.dst}: {pkt.data}")

        # Parse the packet timestamp
        pkt_time = datetime.strptime(pkt.timestamp, "%H:%M:%S.%f")

        # If this is the first packet or the time window has expired, reset the SYN counts and start time
        if start_time is None or pkt_time - start_time > time_window:
            syn_counts.clear()
            start_time = pkt_time

        # If the packet is a TCP SYN packet, increment the SYN count for the destination
        if pkt.protocol == "TCP" and "S" in pkt.data:
            syn_counts[pkt.dst] += 1

        if syn_counts[pkt.dst] > packet_threshold:
            print(f"DOS (SYN flood) attack from: {pkt.src}")